# Save and retrieve jobs

Quantum workflows often take a while to complete and can run over many sessions. Restarting your Python kernel means you'll lose any results stored in memory. To avoid loss of data, you can save results to file and retrieve results of past jobs from IBM Quantum&trade; so your next session can continue where you left off.

## Retrieve jobs from IBM Quantum

IBM Quantum automatically stores results from every job for you to retrieve at a later date. Use this feature to continue quantum programs across kernel restarts and review past results. You can get the ID of a job programmatically through its `job_id` method, or you can see all your submitted jobs and their IDs through the [Jobs dashboard](https://quantum.ibm.com/jobs).

To demonstrate, the following cell sets up the Qiskit Runtime service and selects a backend.

In [5]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, min_num_qubits=2)
sampler = Sampler(backend=backend)

The following cell submits the job and prints its ID.

In [6]:
from qiskit import transpile
job = sampler.run(transpile(qc, backend), shots=10)
my_id = job.job_id()
print(my_id)

co24umeh1p25p1m5nu7g


/Users/qiskit/.venv/python3.11/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:879: UserWarning: Cloud simulators will be deprecated on 15 May 2024. Use the new local testing mode in qiskit-ibm-runtime version 0.22.0 or later to meet your debugging needs.
  warnings.warn(warning_message)


Now that you have the job's ID, you can retrieve it in a different session or at a later date using the `service.job` method.

In [7]:
retrieved_job = service.job(my_id)
retrieved_job.result()

PrimitiveResult([PubResult(data=DataBin<>(meas=BitArray(<shape=(), num_shots=10, num_bits=2>)), metadata={'circuit_metadata': {}})], metadata={'version': 2})

### Programmatically find jobs

If you don't have a job ID and want to find it programmatically rather than visiting the [Jobs dashboard](https://quantum.ibm.com/jobs), you can use the [`QiskitRuntimeService.jobs`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#jobs) method.

The following cell finds any jobs submitted in the last hour. The `created_after` argument must be a [`datetime.datetime`](https://docs.python.org/3.8/library/datetime.html#datetime.datetime) object.

In [8]:
import datetime
one_hour_ago = datetime.datetime.now() - datetime.timedelta(hours=1)

service = QiskitRuntimeService()
service.jobs(created_after=one_hour_ago)

[<RuntimeJob('co24umeh1p25p1m5nu7g', 'sampler')>,
 <RuntimeJob('cr24smydvs8g008jejs0', 'sampler')>]

You can also select by backend, job state, session, and more. For more information, see [`QiskitRuntimeService.jobs`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#jobs) in the API documentation.

## Save results to disk

You may also want to save results to disk. To do this, use Python's built-in JSON library with Qiskit IBM Runtime's encoders.

In [9]:
import json
from qiskit_ibm_runtime import RuntimeEncoder
with open("result.json", "w") as file:
    json.dump(job.result(), file, cls=RuntimeEncoder)

You can then load this array from disk in a separate kernel.

In [10]:
from qiskit_ibm_runtime import RuntimeDecoder
with open("result.json", "r") as file:
    result = json.load(file, cls=RuntimeDecoder)

result[0].data.meas.array  # access measurement data

array([[3],
       [3],
       [3],
       [3],
       [0],
       [3],
       [0],
       [3],
       [0],
       [3]], dtype=uint8)

In [11]:
# Cleanup the file we created (this cell should be hidden from the user)
pathlib.Path('result.json').unlink()